In [1]:
# delete certain id

import sqlite3

con = sqlite3.connect("../../data/landscape-analysis.db")
con.execute("PRAGMA foreign_keys=ON")

# Delete some run ids
# run_ids = [(2,), (3,), (4,)]
run_ids = [(6,)]
con.executemany("DELETE FROM runs WHERE id = ?", run_ids)

# Delete specific algorithm and termination entries
# con.execute("DELETE FROM algorithm WHERE id = ?", (1,))
# con.execute("DELETE FROM termination WHERE id = ?", (1,))

con.commit()
con.close()


In [4]:
# reset column

import sqlite3
con = sqlite3.connect("../../data/landscape-analysis.db")
con.execute("UPDATE termination SET n_max_gen = 10 WHERE id = 1")

con.commit()
con.close()

In [5]:
# print row

import sqlite3
con = sqlite3.connect("../../data/landscape-analysis.db")
for row in con.execute("SELECT * FROM termination WHERE id = 1"):
    print(row)


(1, None, None, None, None, 10, None, None, '9fb6846d7fc941a0df18bf38c9626ccaec3638db')


In [2]:
# clean rows in tables
import sqlite3

db_path = "../../data/landscape-analysis.db"
con = sqlite3.connect(db_path)
cur = con.cursor()

cur.execute("PRAGMA foreign_keys = ON;")

cur.execute("DELETE FROM runs;")
# cur.execute("DELETE FROM algorithm;")
# cur.execute("DELETE FROM termination;")

# reset ID（sqlite used sqlite_sequence to AUTOINCREMENT id）
# cur.execute("DELETE FROM sqlite_sequence WHERE name='runs';")
# cur.execute("DELETE FROM sqlite_sequence WHERE name='algorithm';")
# cur.execute("DELETE FROM sqlite_sequence WHERE name='termination';")

con.commit()
con.close()


In [9]:
cur.execute("SELECT sql FROM sqlite_master WHERE name='runs';")
print(cur.fetchone()[0])

CREATE TABLE runs (
  id INTEGER PRIMARY KEY,
  problem_type TEXT NOT NULL,           -- 'nn' | 'bbob'
  problem_id INTEGER NOT NULL,          -- polymorphic; validated by triggers below
  algorithm_id INTEGER NOT NULL REFERENCES algorithm(id) ON DELETE RESTRICT,
  termination_id INTEGER NOT NULL REFERENCES termination(id) ON DELETE RESTRICT,
  batch_size INTEGER NOT NULL,
  seeds_json TEXT NOT NULL,             -- JSON array of ints
  multi_run INTEGER DEFAULT 0,
  save_history INTEGER DEFAULT 1,
  verbose INTEGER DEFAULT 0,
  result_filename TEXT NOT NULL,
  created_at TEXT DEFAULT (datetime('now')),
  updated_at TEXT
)


In [5]:
con.close()

In [3]:
%cd ../../

import os, json
from pathlib import Path
from src.OptimizerAnalysis.db_helpers import connect, ensure_tables, AlgorithmRow, TerminationRow, _table_cols

DB = os.environ.get("LANDSCAPE_DB", "data/landscape-analysis.db")
con = connect(DB); ensure_tables(con)

# 迁移：保证 termination 表里有 n_max_epochs 列
cols = _table_cols(con, "termination")
if "n_max_epochs" not in cols:
    con.execute("ALTER TABLE termination ADD COLUMN n_max_epochs INTEGER")
    con.commit()
con.close()

POP = 50
defaults = [
    ("SGD",   {"optimizer": "adam", "lr": 0.01}),
    ("CMAES", {"popsize": POP, "sigma": 0.3}),
    ("DE",    {"CR": 0.9, "F": 0.5, "pop_size": POP}),
    ("PSO",   {"pop_size": POP}),
    ("GA",    {"pop_size": POP}),
    ("ES",    {"pop_size": POP, "n_offsprings": 2*POP, "rule": 1.0/7.0}),
    ("SRES",  {"pop_size": POP, "n_offsprings": 2*POP, "rule": 1.0/7.0, "gamma": 0.85, "alpha": 0.2}),
    ("ISRES", {"pop_size": POP, "n_offsprings": 2*POP, "rule": 1.0/7.0, "gamma": 0.85, "alpha": 0.2}),
]

ids = {}
for name, params in defaults:
    aid = AlgorithmRow(name=name.upper(), hyperparameter_problem=False, non_default_params=params).write_to_db(DB)
    ids[name.upper()] = aid

term_id = TerminationRow(n_max_epochs=100).write_to_db(DB)

print("ALGO_IDS:", ids)
print("TERM_ID:", term_id)

Path("algo_ids.json").write_text(json.dumps(ids, indent=2))
Path("algo_ids.txt").write_text(" ".join(str(v) for v in ids.values()))
Path("term_id.txt").write_text(str(term_id))

/scratch/wx2178/.conda/envs/cns/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/scratch/wx2178/SNN/DarwinNeuron
ALGO_IDS: {'SGD': 1, 'CMAES': 2, 'DE': 3, 'PSO': 4, 'GA': 5, 'ES': 6, 'SRES': 7, 'ISRES': 8}
TERM_ID: 1


1